In [1]:
################################################################################
#Using calculated isotopomer paths, two main approaches of trimming will be used
#1) Using circadian EC hits from Xu, 2011 and Gill, 2015, to pull out any rows
#...which contain any ECs, and the number of ECs in a row given will also be added
#
#2) Gibbs energies have been estimated in Tepper, 2013, which will be used to 
#...predict path thermodynamics
#
#Note this is an optional script, and an example of how one may go about trimming
#.. excess results through other means, such as previously published 'omic data
#.. or reaction thermodynamics
#
#This script should be used as a guide, as the trimming procedures can be very
#.. dependent on context and experimental objectives
################################################################################

In [2]:
import numpy
import pandas
import os
import sys
import re
import itertools
import csv
from itertools import chain, repeat
from collections import defaultdict
import dill as pickle
from pathos.helpers import mp

In [3]:
#Read in a list of significant circadian ECs from Xu et al, 2011 and Gill et al, 2015
SigECList=pandas.read_csv('Xu2011Gill2015_ECs_Cyclers.csv',header=None)
ECList=list(SigECList[0])

#Edit names to match the format of the PathSearch results
ECList=['EC-'+x for x in ECList]

FileNotFoundError: File b'Xu2011Gill2015_ECs_Cyclers.csv' does not exist

In [23]:
#Read in the list of estimated Gibbs energies for all annotated enzymes, from Tepper et al., 2013
GibbsList=pandas.read_csv('Tepper2013_GibbsECs.csv',skiprows=1)
GibbsECList=['EC-'+x for x in list(GibbsList['ec number'])]
#Note this will take the last delta G value in a list of duplicated ECs

GibbsListDict=dict(zip(GibbsECList,GibbsList['standart Gibbs energy (kJ/mol)']))

In [24]:
#Read in the results of the isotopologue file of interest
#GlnDF=pandas.read_csv('Glutamine M+2_Paths_13Rxns.csv',header=None,error_bad_lines=False)
#SerDF=pandas.read_csv('Serine M+3_Paths_9Rxns.csv',header=None,error_bad_lines=False)
#GSHDF=pandas.read_csv('Glutathione Reduced M+2_Paths_9Rxns.csv',header=None,error_bad_lines=False)

In [26]:
#From the fly studies, drop any paths which don't have circadian enzymes
#NOTE!# This step is optional, it is entirely possible that pathway activities may be cyclic without a circadian enzyme (for more detail, see Thurley et al, PNAS, 2017)

def KeggECTrimmer(RowItem,RxnNumber):
    #Convert the stringy EC item into a list of one/multiple ECs
    ListyECs=[re.sub('\[|\'|\]','',x) for x in re.split('\, ',RowItem)]
    
    #if any item across the row is found in the circadian enzyme list
    if any([y for y in ListyECs if y in KeggECList]):
        RowItem=[y for y in ListyECs if y in KeggECList]
        return(RowItem)
    else:
        return(RxnNumber)

In [27]:
#Find ECs for reporting - if there isn't an EC (say only a Metacyc RXN is left), it can be dropped - of course the 'RXN' format in MetaCyc may represent real reactions, but if it isn't connection to an EC the interpretation is more difficult
def MatchFlyKeggECs(PathMatrix):
    DropIndices=[]
    for rxn in range(len(PathMatrix)):
        ECMatchList=[KeggECTrimmer(z,rxn) for z in PathMatrix[rxn:rxn+1].values[0] if type(z) is str and 'EC' in z]
        if any([x==rxn for x in ECMatchList]):
            DropIndices.append(rxn)
            
    #Store indicies where there isn't a RXN -> EC conversion
    PathMatrix=PathMatrix.drop(DropIndices)
    PathMatrix=PathMatrix.reset_index(drop=True)
    return(PathMatrix)

In [28]:
#Sum up the gibbs energies for a given metabolic path
def SumGibbsAcrossRow(PathMatrix):
    MiniGibbList=[]
    for items in PathMatrix:

        try:
            if any([GibbsListDict[x] for x in GibbsECList if x in re.split('\'|\,',items)]):
                
                MiniGibbList.append(min([GibbsListDict[x] for x in GibbsECList if x in re.split('\'|\,',items)]))
        except:
            pass
        
    PathMatrix['Gibbs Sum']=sum(MiniGibbList)
    return(PathMatrix)

In [29]:
#Match ECs in isotopomer paths to significant EC hits from Xu,2011 and Gill,2015
def CircadianECCounter(PathMatrix):
    
    TempECList=[]
    
    for items in PathMatrix:
        
        try:
            
            if any([x for x in ECList if x in re.split('\'|\,',items)])==True:
                    
                    #Add the EC to the list
                TempECList.extend([x for x in ECList if x in re.split('\'|\,',items)])
                    
        except:
            pass
    
    #Only unique items in the ECList
    TempECList=list(set(TempECList))
    
    PathMatrix['Number of Circadian ECs']=len(TempECList)
    PathMatrix['Circadian ECs']=TempECList
    
    return(PathMatrix)

In [30]:
#Function to find ECs in the PathMatrix which match to Xu,2011 and Gill,2015 circadian hits
#This function is currently very slow for large data frames
#Converted to apply functions to take out loops and still slow, some optimization may be required
def MatchCircadianECHitsAndGetGibbs(PathMatrix):
    
    PathMatrix=PathMatrix.reset_index(drop=True)
    
    #Drop duplicates
    PathMatrix=PathMatrix.drop_duplicates()
    
    PathMatrix=PathMatrix.reset_index(drop=True)
    
    #ECs added
    PathMatrixECs=PathMatrix.apply(CircadianECCounter,axis=1)
    
    #Caculate Gibbs across each path
    PathMatrix=PathMatrix.apply(SumGibbsAcrossRow,axis=1)

    
    PathMatrix['Number of Circadian ECs']=PathMatrixECs['Number of Circadian ECs']
    PathMatrix['Circadian ECs']=PathMatrixECs['Circadian ECs']
    
    #Drop rows which don't match any ECs (optional)
    PathMatrixDrop=PathMatrix[PathMatrix['Number of Circadian ECs']!=0]
    #PathMatrixDrop=PathMatrix
    
    PathMatrixDrop=PathMatrixDrop.reset_index(drop=True)
    
    ##Sort by number of matches
    ##PathMatrixDrop=PathMatrixDrop.sort_values('Gibbs Sum',ascending=True)
    
    #Drop certain unwanted items
    PathMatrixDrop=DropSpecifics(PathMatrixDrop)
    
    PathMatrixDrop=PathMatrixDrop.reset_index(drop=True)
    
    return(PathMatrixDrop)


In [31]:
#Drop certain metabolites that aren't relevant or don't match beyond a PubChem ID
#More features can be added here depending on the organism, experimenters' needs etc..
#This is very slow, may need a new way to do this
def DropSpecifics(PathMatrix):
    
    #Regex for certain ECs
    EC1=re.compile('\[\'EC-4.1.2.22\'\]')
    EC2=re.compile('\[\'EC-4.1.2.9\'\]')
    EC3=re.compile('\'EC-4.1.1.39\'')
    
    DropIndex=[]
    for rxn in range(len(PathMatrix)):
        if any([x for x in PathMatrix[rxn:rxn+1].values[0] if type(x) is str and 'Sorbose' in x]):
            DropIndex.append(rxn)
        if any([x for x in PathMatrix[rxn:rxn+1].values[0] if type(x) is str and 'AC1L1ARE' in x]):
            DropIndex.append(rxn)
        if any([x for x in PathMatrix[rxn:rxn+1].values[0] if type(x) is str and EC1.search(x)]):
            DropIndex.append(rxn)
        if any([x for x in PathMatrix[rxn:rxn+1].values[0] if type(x) is str and EC2.search(x)]):
            DropIndex.append(rxn)
        if any([x for x in PathMatrix[rxn:rxn+1].values[0] if type(x) is str and EC3.search(x)]):
            DropIndex.append(rxn)
    DropIndex=list(set(DropIndex))
    PathMatrix=PathMatrix.drop(DropIndex)
    PathMatrix=PathMatrix.reset_index(drop=True)
    
    return(PathMatrix)

In [32]:
#Making DropSpecifics parallel
PathMatrixSplit=list(numpy.array_split(SerDF,16))
    
pooler=mp.Pool(16)
        
with open('Trimmed_Paths.csv','a') as fp: #originally 'w' - but append for looping through shorter path lengths
    #written out to file as its being built
        for result in pooler.imap(MatchCircadianECHitsAndGetGibbs,PathMatrixSplit):
            #Each result is a Pandas Object, so write it to csv
            result.to_csv(fp,index=False,header=False)

pooler.close()
pooler.join()


In [33]:
##Read back in to sort
##Sort by number of matches
SerDF=pandas.read_csv('Trimmed_Paths.csv',header=None)
SerDF=SerDF.sort_values(SerDF.columns[-3],ascending=True)
SerDF=SerDF.drop_duplicates()
SerDF=SerDF.reset_index(drop=True)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,Serine M+3,['EC-4.3.1.17'],Pyruvate M+3,['EC-2.7.1.40'],phosphoenolpyruvate M+3,['EC-4.2.1.11'],2-phosphoglyceric acid M+3,['EC-5.4.2.12'],3-Phosphoglycerate M+3,['EC-2.7.2.3'],...,Glycerophosphoric acid M+3,['EC-4.1.2.13'],"1,6-di-o-phosphonohexopyranose M+6",['EC-2.7.1.11'],Hexose-6-Phosphate M+6,"['EC-2.7.1.1', 'EC-2.7.1.147']",alpha-D-Glucose M+6,-47.733063,1,['EC-2.7.1.40']
1,Serine M+3,['EC-4.3.1.17'],Pyruvate M+3,['EC-2.7.1.40'],phosphoenolpyruvate M+3,['EC-4.2.1.11'],2-phosphoglyceric acid M+3,['EC-5.4.2.12'],3-Phosphoglycerate M+3,['EC-2.7.2.3'],...,Glycerophosphoric acid M+3,['EC-4.1.2.13'],"1,6-di-o-phosphonohexopyranose M+6",['EC-2.7.1.10'],Hexose-1-Phosphate M+6,['EC-3.1.3.10'],alpha-D-Glucose M+6,-30.187252,1,['EC-2.7.1.40']
2,Serine M+3,['EC-4.3.1.17'],Pyruvate M+3,['EC-2.7.1.40'],phosphoenolpyruvate M+3,['EC-4.2.1.11'],2-phosphoglyceric acid M+3,['EC-5.4.2.12'],3-Phosphoglycerate M+3,['EC-2.7.2.3'],...,Glycerophosphoric acid M+3,['EC-4.1.2.13'],"1,6-di-o-phosphonohexopyranose M+6",['EC-2.7.1.10'],Hexose-1-Phosphate M+6,['EC-2.7.1.6'],alpha-D-Glucose M+6,-21.970038,1,['EC-2.7.1.40']
3,Serine M+3,['EC-4.3.1.17'],Pyruvate M+3,['EC-2.7.1.40'],phosphoenolpyruvate M+3,['EC-4.2.1.11'],2-phosphoglyceric acid M+3,['EC-5.4.2.12'],3-Phosphoglycerate M+3,['EC-2.7.2.3'],...,Glycerophosphoric acid M+3,['EC-4.1.2.13'],"1,6-di-o-phosphonohexopyranose M+6",['EC-2.7.1.10'],Hexose-1-Phosphate M+6,['EC-2.4.1.231'],alpha-D-Glucose M+6,-11.360571,1,['EC-2.7.1.40']
4,Serine M+3,['EC-2.6.1.45'],3-Hydroxypyruvic acid M+3,"['EC-1.1.1', 'EC-1.1.1.29', 'EC-1.1.1.81']",Glycerate M+3,['EC-2.7.1.31'],3-Phosphoglycerate M+3,['EC-2.7.2.3'],"1,3-Bisphosphoglycerate M+3",['EC-1.2.1.12'],...,"1,6-di-o-phosphonohexopyranose M+6",['EC-2.7.1.11'],Hexose-6-Phosphate M+6,['EC-5.4.2.5'],Hexose-1-Phosphate M+6,['EC-3.1.3.10'],alpha-D-Glucose M+6,-0.582832,0,[]
5,Serine M+3,['EC-2.6.1.45'],3-Hydroxypyruvic acid M+3,"['EC-1.1.1', 'EC-1.1.1.29', 'EC-1.1.1.81']",Glycerate M+3,['EC-2.7.1.31'],3-Phosphoglycerate M+3,['EC-2.7.2.3'],"1,3-Bisphosphoglycerate M+3",['EC-1.2.1.12'],...,Dihydroxyacetone Phosphate M+3,['EC-4.1.2.13'],"1,6-di-o-phosphonohexopyranose M+6",['EC-2.7.1.11'],Hexose-6-Phosphate M+6,"['EC-2.7.1.1', 'EC-2.7.1.147']",alpha-D-Glucose M+6,7.158978,0,[]
6,Serine M+3,['EC-2.6.1.45'],3-Hydroxypyruvic acid M+3,"['EC-1.1.1', 'EC-1.1.1.29', 'EC-1.1.1.81']",Glycerate M+3,['EC-2.7.1.31'],3-Phosphoglycerate M+3,['EC-2.7.2.3'],"1,3-Bisphosphoglycerate M+3",['EC-1.2.1.12'],...,"1,6-di-o-phosphonohexopyranose M+6",['EC-2.7.1.11'],Hexose-6-Phosphate M+6,['EC-5.4.2.5'],Hexose-1-Phosphate M+6,['EC-2.7.1.6'],alpha-D-Glucose M+6,7.634382,0,[]
7,Serine M+3,['EC-2.6.1.45'],3-Hydroxypyruvic acid M+3,"['EC-1.1.1', 'EC-1.1.1.29', 'EC-1.1.1.81']",Glycerate M+3,['EC-2.7.1.31'],3-Phosphoglycerate M+3,['EC-2.7.2.3'],"1,3-Bisphosphoglycerate M+3",['EC-1.2.1.12'],...,"1,6-di-o-phosphonohexopyranose M+6",['EC-2.7.1.11'],Hexose-6-Phosphate M+6,['EC-5.4.2.5'],Hexose-1-Phosphate M+6,['EC-2.4.1.231'],alpha-D-Glucose M+6,18.243849,0,[]
8,Serine M+3,['EC-2.6.1.45'],3-Hydroxypyruvic acid M+3,"['EC-1.1.1', 'EC-1.1.1.29', 'EC-1.1.1.81']",Glycerate M+3,['EC-2.7.1.31'],3-Phosphoglycerate M+3,['EC-2.7.2.3'],"1,3-Bisphosphoglycerate M+3",['EC-1.2.1.12'],...,"1,6-di-o-phosphonohexopyranose M+6",['EC-2.7.1.11'],Hexose-6-Phosphate M+6,['EC-5.4.2.6'],Hexose-1-Phosphate M+6,['EC-2.4.1.64'],alpha-D-Glucose M+6,18.243849,0,[]
9,Serine M+3,['EC-2.6.1.45'],3-Hydroxypyruvic acid M+3,"['EC-1.1.1', 'EC-1.1.1.29', 'EC-1.1.1.81']",Glycerate M+3,['EC-2.7.1.31'],3-Phosphoglycerate M+3,['EC-2.7.2.3'],"1,3-Bisphosphoglycerate M+3",['EC-1.2.1.12'],...,"1,6-di-o-phosphonohexopyranose M+6",['EC-2.7.1.10'],Hexose-1-Phosphate M+6,['EC-5.4.2.5'],Hexose-6-Phosphate M+6,"['EC-2.7.1.1', 'EC-2.7.1.147']",alpha-D-Glucose M+6,20.410832,0,[]
